In [1]:
import os
import sys
import platform
from typing import List, Dict, Optional, Union, Tuple

In [2]:
test_dir = "img.test.dir/"
test_dir = os.path.abspath(test_dir)

In [3]:
mod_path = "../bin"

In [4]:
# os.listdir(mod_path)
sys.path.append(mod_path)

In [5]:
import convert_source as cs

In [6]:
c_file = "../config.default.yml"

In [7]:
[search_dict, \
    exclude_list, \
        meta_dict ] = cs.read_config(config_file=c_file,verbose=True)

Initialized parameters from configuration file
Categorizing search terms
Exclusion option implemented
Including additional settings for metadata


In [8]:
import utils.img_dir as id

In [9]:
# os.listdir(test_dir)
[dir_list, id_list] = id.img_dir_list(directory=test_dir,verbose=False)

In [10]:
dir_list[0]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\DICOM\\20211701'

In [11]:
if 'windows' in platform.platform().lower():
    path_sep = "\\"
else:
    path_sep = "/"

In [12]:
# path_sep = "/"

In [13]:
# Isolate subject and session ID
os.path.dirname(os.path.dirname(dir_list[0].replace(test_dir + path_sep,""))).split(sep="-")

['001', '001']

In [14]:
# Isolate subject and session ID
# os.path.dirname(os.path.basename(dir_list[0].replace(test_dir + path_sep,"")))
os.path.dirname(os.path.dirname(dir_list[0]))

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001'

In [16]:
[sub, ses] = os.path.dirname(os.path.dirname(dir_list[0].replace(test_dir + path_sep,""))).split(sep="-")

In [17]:
sub

'001'

In [18]:
os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            dir_list[9].replace(test_dir + path_sep,"")))).split(sep="-")

['900XXT5']

In [19]:
os.path.dirname(dir_list[9]).replace(test_dir + path_sep,"").split(sep="-")

['900XXT5\\DICOM\\20211701']

In [20]:
class SubDataInfo():
    '''Class instance that creates a data object that organizes a subject's 
    identification (ID) number, session ID number, and the 
    path to the image data directory. This information is then stored for 
    each separate class instance, and can be accessed as shown in the example
    usage.
    
    Usage example:
        >>> sub_info = SubDataInfo(sub="002",
                                    data="<path/to/img/data>",
                                    ses="001")
        >>> sub_info.sub
        "002"
        >>> 
        >>> sub_info.ses
        "001"
    '''

    def __init__(self,
                 sub: str,
                 data: str,
                 ses: Optional[str] = None):
        '''Init doc-string for the 'SubDataInfo' class. 
        
        Arguments:
            sub: Subject ID.
            data: Path to image data directory.
            ses: Session ID.
        '''
        self.sub: str = sub
        self.data: str = data
        if ses:
            self.ses: str = ses
        else:
            self.ses: str = ""

In [21]:
def collect_info(parent_dir: str) -> List[SubDataInfo]:
    '''Collects image data information for each subject for a study, 
    provided there exists some parent directory.

    Usage example:
        >>> data = collect_info("<parent/directory>")
        >>>
        >>> data[0].sub
        "<subject_ID>"
        >>> 
        >>> data[0].data
        "<path/to/data>"
        >>> 
        >>> data[0].ses
        "<session_ID>"
    
    Arguments:
        parent_dir: Parent directory that contains each subject.
        
    Returns:
        List/Array of SubDataInfo objects that corresponds to a subject ID, 
            session ID, and path to medical image directory.
    '''
    
    data: List = []

    if 'windows' in platform.platform().lower():
        path_sep = "\\"
    else:
        path_sep = "/"
    
    # Get image directory information
    [dir_list, id_list] = id.img_dir_list(directory=parent_dir,
                                          verbose=False)

    # Iterate through each subject image directory
    for img in dir_list:
        # Set empty variables
        sub = ""; ses = ""
        
        # Get subject and session ID from file path
        try:
            [sub, ses] = img.replace(parent_dir + path_sep,"").split(sep=path_sep)[0].split(sep="-")
        except ValueError:
            ses = ""
            sub = img.replace(parent_dir + path_sep,"").split(sep=path_sep)[0]
                
    
        # Collect and organize each subjects' session and data
        sub_info = SubDataInfo(sub=sub,
                               data=img,
                               ses=ses)
        
        data.append(sub_info)
        
        # Clear variables
        del sub, ses
        
    return data

In [93]:
# def collect_info(parent_dir: str) -> List[SubDataInfo]:
#     '''Collects image data information for each subject for a study, 
#     provided there exists some parent directory.
    
#     Arguments:
#         parent_dir: Parent directory that contains each subject.
        
#     Returns:
#         List/Array of SubDataInfo objects that corresponds to a subject ID, 
#             session ID, and path to medical image directory.
#     '''
    
#     data: List = []

#     if 'windows' in platform.platform().lower():
#         path_sep = "\\"
#     else:
#         path_sep = "/"
    
#     # Get image directory information
#     [dir_list, id_list] = id.img_dir_list(directory=parent_dir,
#                                           verbose=False)

#     # Iterate through each subject image directory
#     for img in dir_list:
#         # Set empty variables
#         sub = ""; ses = ""
        
#         # Edge case 2
#         try:
#             [sub, ses] = os.path.dirname(
#                 os.path.dirname(
#                     os.path.dirname(
#                         img.replace(parent_dir + path_sep,"")))).split(sep="-")
#         except ValueError:
#             ses = ""
#             sub = os.path.dirname(
#                 os.path.dirname(
#                     os.path.dirname(
#                         img.replace(parent_dir + path_sep,""))))#.split(sep="-")
        
#         # Edge case 1
#         if len(sub) == 0:
#             try:
#                 [sub, ses] = os.path.dirname(
#                     os.path.dirname(
#                         img.replace(parent_dir + path_sep,""))).split(sep="-")
#             except ValueError:
#                 ses = ""
#                 sub = os.path.dirname(
#                     os.path.dirname(
#                         img.replace(parent_dir + path_sep,"")))#.split(sep="-")
        
#         # Base case
#         if len(sub) == 0:
#             try:
#                 [sub, ses] = os.path.dirname(
#                     img.replace(parent_dir + path_sep,"")).split(sep="-")
#             except ValueError:
#                 ses = ""
#                 sub = os.path.dirname(
#                     img.replace(parent_dir + path_sep,""))#.split(sep="-")
                
    
#         # Collect and organize each subjects' session and data
#         sub_info = SubDataInfo(sub=sub,
#                                data=img,
#                                ses=ses)
        
#         data.append(sub_info)
        
#         # Clear variables
#         del sub, ses
        
#     return data

In [58]:
t = {}

SyntaxError: keyword can't be an expression (<ipython-input-62-2f4a265e7b9c>, line 1)

In [57]:
t

{'sub': '001'}

In [20]:
s = SubDataInfo("001",dir_list[0],"001")

In [24]:
s.data

'C:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\DICOM\\20211701'

In [25]:
t = []; t.append(s)

In [26]:
s = SubDataInfo("002",dir_list[1],"001")

In [28]:
s.data

'C:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI'

In [29]:
t.append(s)

In [32]:
t[1].sub

'002'

In [22]:
data = collect_info(parent_dir=test_dir)

In [23]:
len(data)

36

In [24]:
data[1].data

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI'

In [25]:
data[1].ses

'001'

In [26]:
data[1].sub

'001'

In [27]:
for i in range(0,len(data)):
    try:
        print(f"sub-{data[i].sub}_ses-{data[i].ses}")
        # print(f"sub-{data[i].sub}")
    except AttributeError:
        pass

sub-001_ses-001
sub-001_ses-001
sub-001_ses-001
sub-002_ses-001
sub-002_ses-001
sub-002_ses-001
sub-003_ses-001
sub-003_ses-001
sub-003_ses-001
sub-900XXT5_ses-
sub-900XXT5_ses-
sub-900XXT5_ses-
sub-901XXP5_ses-
sub-901XXP5_ses-
sub-901XXP5_ses-
sub-902XXY8_ses-
sub-902XXY8_ses-
sub-902XXY8_ses-
sub-C01_ses-001
sub-C01_ses-001
sub-C01_ses-001
sub-C02_ses-001
sub-C02_ses-001
sub-C02_ses-001
sub-C03_ses-001
sub-C03_ses-001
sub-C03_ses-001
sub-P01_ses-001
sub-P01_ses-001
sub-P01_ses-001
sub-P02_ses-001
sub-P02_ses-001
sub-P02_ses-001
sub-P03_ses-001
sub-P03_ses-001
sub-P03_ses-001


In [11]:
dir_list[0]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\DICOM\\20211701'

In [12]:
dir_list[1]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI'

In [13]:
dir_list[2]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\PAR REC'

In [14]:
dir_list[9]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\900XXT5\\DICOM\\20211701\\NAME^ 20201701'

In [15]:
from pathlib import Path

In [24]:
# Edge case 1 - DICOM
os.path.basename(Path(dir_list[0]).parents[1])

'001-001'

In [26]:
# Base case [likely NIFTI and PAR REC]
os.path.basename(Path(dir_list[1]).parents[0])

'001-001'

In [29]:
# Edge case 2 - DICOMs in child directory
os.path.basename(Path(dir_list[9]).parents[2])

'900XXT5'

In [35]:
os.path.basename(Path(dir_list[9]).parents[2])

'900XXT5'

In [38]:
os.path.basename(dir_list[1])

'NIFTI'

In [47]:
dir_list[1]

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI'

In [42]:
os.path.basename(Path(os.path.dirname(dir_list[0])).parents[0])

'001-001'

In [48]:
Path(dir_list[1]).parents[0]

WindowsPath('c:/Users/smart/Desktop/projects/cs_med_data/convert_source/tests/img.test.dir/001-001')

In [62]:
Path(os.path.dirname(dir_list[1])).parents[0]

WindowsPath('c:/Users/smart/Desktop/projects/cs_med_data/convert_source/tests/img.test.dir')

In [66]:
str(Path(dir_list[1])).split("\\")

['c:',
 'Users',
 'smart',
 'Desktop',
 'projects',
 'cs_med_data',
 'convert_source',
 'tests',
 'img.test.dir',
 '001-001',
 'NIFTI']

In [74]:
str(Path(dir_list[0])).split("\\")[-3]

'001-001'

In [73]:
str(Path(dir_list[9])).split("\\")[-4]

'900XXT5'

In [81]:
os.path.dirname(os.path.dirname(os.path.dirname(dir_list[0])))

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir'

In [91]:
str(Path(dir_list[0]).parent).split("\\")

['c:',
 'Users',
 'smart',
 'Desktop',
 'projects',
 'cs_med_data',
 'convert_source',
 'tests',
 'img.test.dir',
 '001-001',
 'DICOM']

In [100]:
Path(dir_list[0].replace(test_dir + "\\","")).parent

WindowsPath('001-001/DICOM')

In [101]:
dir_list[0].replace(test_dir + "\\","")

'001-001\\DICOM\\20211701'

In [102]:
dir_list[1].replace(test_dir + "\\","")

'001-001\\NIFTI'

In [104]:
path_sep = "\\"

In [109]:
dir_list[9].replace(test_dir + "\\","").split(path_sep)[0].split("-")

['900XXT5']

In [108]:
dir_list[0].replace(test_dir + "\\","").split(path_sep)[0].split("-")

['001', '001']

In [110]:
dir_list[1].replace(test_dir + "\\","").split(path_sep)[0].split("-")

['001', '001']